In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(r'C:\Users\nahue\Searches\Escritorio\CARRERA HENRY\Primer_proyecto_individual\Datasets\df_para_recomendacion.csv', low_memory=False)

In [3]:
df.head(3)

,vote_average,title
0,7.7,Toy Story
1,6.9,Jumanji
2,6.5,Grumpier Old Men


### Para hacer la función tuve que investigar varias cosas:
Vectorización TF-IDF: es una técnica comúnmente utilizada en el procesamiento del lenguaje natural (NLP) para convertir un conjunto de documentos de texto en una matriz de características numéricas. TF-IDF significa Term Frequency-Inverse Document Frequency (Frecuencia de Término-Frecuencia Inversa de Documento).

#### Frecuencia de Término (TF):

La Frecuencia de Término mide la frecuencia con la que una palabra aparece en un documento en comparación con el total de palabras en ese documento. Se calcula como:

(Número de veces que el término aparece en el documento)/(Número total de términos en el documento)

Ej: "El gato come pescado" el TF de cada palabra es 0.25 ya que cada termino aparece una vez y el total de terminos son cuatro (1/4)
    "El perro come carne" el TF es igual al anterior.

#### Frecuencia Inversa de Documento (IDF):

La Frecuencia Inversa de Documento mide la importancia de una palabra en el conjunto de documentos. La idea es que las palabras que aparecen en muchos documentos no son muy útiles para diferenciar entre documentos. Se calcula como:

log({Número total de documentos}/{Número de documentos que contienen el término})

Ej: Si tomamos gato, perro, pescado y carne el IDF es log 2/1 = 0.3.
    Si tomamos El y come el IDF es log 2/2 = 0

Finalmente se calcula TF-IDF multiplicando TF x IDF:
Ej: para gato, perro, pescado y carne: 0.25 * 0.3 = 0.075,
    mientras que para El y come: 0.25 * 0 = 0

Lo que se concluye es que a mayor aparición de una misma palabra en distintos documentos, menos relevante es para caracterizar algún contenido.

In [20]:
#Ejemplo de la aplicación en la función para entender los pasos:

#Invento una base de datos pequeña:
pelis = {'titulo': ["The lion king", "The book of the jungle", "The white lion"]}
df_pelis = pd.DataFrame(pelis)

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

#TfidfVectorizer: Esta clase convierte una colección de documentos en una matriz de características TF-IDF, que nos ayuda a medir 
#la importancia de las palabras en los títulos de las películas.

tfidf_pelis = TfidfVectorizer(stop_words='english') #Creamos una instancia de TfidfVectorizer e indicamos que ignore las palabras comunes en inglés (the, of) con stop_words
tfidf_pelis_matrix = tfidf_pelis.fit_transform(df_pelis['titulo'])#Aplicamos la vectorización TF-IDF a los títulos de las películas
tfidf_pelis_matrix.toarray()
#El resultado es una matriz en la que cada fila representa un título y cada columna representa una palabra, con valores que indican 
#la importancia de cada palabra en el título correspondiente.

array([[0.        , 0.        , 0.79596054, 0.60534851, 0.        ],
       [0.70710678, 0.70710678, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.60534851, 0.79596054]])

In [25]:
#Analisis de similitud del coseno
from sklearn.metrics.pairwise import linear_kernel #linear_kernel calcula la similitud del coseno entre vectores

cosine_sim = linear_kernel(tfidf_pelis_matrix, tfidf_pelis_matrix)
cosine_sim

#La primer fila y columna es la misma pelicula, y lo mismo con las filas 2 y 3. Por eso se forma la diagonal con 1.
#Donde la similitud es 1, es el mismo título; donde es 0 los títulos no comparten nada; el valor 0.366 es porque comparten la palabra 'lion'

array([[1.        , 0.        , 0.36644682],
       [0.        , 1.        , 0.        ],
       [0.36644682, 0.        , 1.        ]])

## Así queda la función para recomendación de películas

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.preprocessing import StandardScaler
import numpy as np

# Vectorizamos los títulos
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matriz = tfidf.fit_transform(df['title'].str.lower())

# Realizamos el análisis de similitud de los títulos
sim_coseno = linear_kernel(tfidf_matriz, tfidf_matriz)

# Normalizamos la columna 'vote_average' para facilitar la combinación de las métricas de títulos y votos
scaler = StandardScaler()
votos_norm = scaler.fit_transform(df[['vote_average']])

# Función para recomendar películas basadas en un índice de película dada:

def recomendacion(titulo: str):
    titulo = titulo.lower()
    
    idx = df[df['title'].str.lower() == titulo].index[0]#Encontramos el índice de la película cuyo título coincide con el proporcionado.
    sim_scores = list(enumerate(sim_coseno[idx]))#se crea una lista de tuplas donde cada tupla contiene un índice de película y su puntuación de similitud de títulos con la película dada.

    # Para incluir la similitud de votos en el cálculo:
    # En este paso ajustamos las puntuaciones de similitud para incluir tanto la similitud de títulos como la similitud de votos.
    # sim_scores se actualiza para que cada tupla contenga el índice de la película y una puntuación combinada.
    sim_scores = [(i, score * 0.5 + votos_norm[i] * 0.5 + np.random.normal(0, 0.01)) for i, score in sim_scores]#score * 0.5 y votos_norm[i] * 0.5 ponderan igualmente la similitud de títulos y la puntuación de votos
    #El np.random.normal añade ruido aleatorio a la recomendacion ya que sin él, la función recomienda prácticamente las mismas películas
    
    
    #Para ordenar las películas por similitud:
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    #Las películas se ordenan por sus puntuaciones combinadas en orden descendente, de modo que las películas más similares 
    #(tanto en título como en votos) estén al principio de la lista.
   
    sim_scores = sim_scores[1:6]  # así obtenemos las 5 películas más similares omitiendo la primera ya que es la misma que se usó como referencia
    pelicula_indices = [i[0] for i in sim_scores]#extraemos los indices de peliculas similares.

    return df['title'].iloc[pelicula_indices].values.tolist() #devuelve los títulos de las películas recomendadas, utilizando los índices almacenados en pelicula_indices

titulo_pelicula = 'assassins'
recomendaciones = recomendacion(titulo_pelicula)
print("Películas recomendadas:", recomendaciones)


Películas recomendadas: ['At Ellen’s Age', 'My Foolish Heart', "If God Is Willing and da Creek Don't Rise", 'Truckfighters', 'Christopher Titus: Angry Pursuit of Happiness']
